# Classifying How LGUs are Fairing Against Covid-19

This notebooks aims to classify the LGUs in the Philippines, particularly in NCR if they are winning, nearly there and losing, against the fight to Covid-19.

In [1]:
import numpy as np
import pandas as pd
# import tensorflow as tf

In [2]:
#use a python file containing load data function
from covid_load_data import load_data

In [3]:
df = load_data('confirmed', update=False) #for faster runtime, set to false. update only once necessary
df = (df- df.shift(1)).iloc[1:].transpose().reset_index()
df.head()

,Country/Region,2020-01-23 00:00:00,2020-01-24 00:00:00,2020-01-25 00:00:00,2020-01-26 00:00:00,2020-01-27 00:00:00,2020-01-28 00:00:00,2020-01-29 00:00:00,2020-01-30 00:00:00,2020-01-31 00:00:00,...,2020-05-02 00:00:00,2020-05-03 00:00:00,2020-05-04 00:00:00,2020-05-05 00:00:00,2020-05-06 00:00:00,2020-05-07 00:00:00,2020-05-08 00:00:00,2020-05-09 00:00:00,2020-05-10 00:00:00,2020-05-11 00:00:00
0,Afghanistan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,134.0,235.0,190.0,330.0,168.0,171.0,215.0,255.0,369.0,285.0
1,Albania,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.0,6.0,8.0,17.0,12.0,10.0,8.0,6.0,12.0,4.0
2,Algeria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,141.0,179.0,174.0,190.0,159.0,185.0,187.0,189.0,165.0,168.0
3,Andorra,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,2.0,1.0,0.0,1.0,0.0,2.0,1.0,0.0
4,Angola,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,1.0,0.0,0.0,7.0,0.0,2.0,0.0


In [7]:
labels = pd.read_csv('datafiles\country_labels.csv')
labels.columns

Index(['Country', 'Label'], dtype='object')

In [10]:
df.merge(labels, how='inner', left_on='Country/Region', right_on='Country')

,Country/Region,2020-01-23 00:00:00,2020-01-24 00:00:00,2020-01-25 00:00:00,2020-01-26 00:00:00,2020-01-27 00:00:00,2020-01-28 00:00:00,2020-01-29 00:00:00,2020-01-30 00:00:00,2020-01-31 00:00:00,...,2020-05-04 00:00:00,2020-05-05 00:00:00,2020-05-06 00:00:00,2020-05-07 00:00:00,2020-05-08 00:00:00,2020-05-09 00:00:00,2020-05-10 00:00:00,2020-05-11 00:00:00,Country,Label
0,Albania,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.0,17.0,12.0,10.0,8.0,6.0,12.0,4.0,Albania,Nearly
1,Andorra,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,0.0,1.0,0.0,2.0,1.0,0.0,Andorra,Winning
2,Argentina,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,104.0,133.0,188.0,163.0,240.0,165.0,258.0,244.0,Argentina,Losing
3,Australia,0.0,0.0,0.0,4.0,1.0,0.0,1.0,3.0,0.0,...,25.0,28.0,19.0,19.0,5.0,21.0,9.0,22.0,Australia,Winning
4,Austria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,24.0,29.0,34.0,68.0,22.0,59.0,38.0,11.0,Austria,Winning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,Ukraine,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,418.0,366.0,487.0,507.0,504.0,515.0,522.0,416.0,Ukraine,Losing
88,United Kingdom,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,3990.0,4411.0,6116.0,5618.0,4652.0,3896.0,3924.0,3883.0,United Kingdom,Losing
89,Uruguay,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,13.0,3.0,11.0,10.0,8.0,5.0,4.0,Uruguay,Nearly
90,Uzbekistan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,40.0,18.0,26.0,65.0,27.0,24.0,69.0,68.0,Uzbekistan,Nearly
